In [1]:
import os , sys
sys.path.append(os.path.abspath('../scripts'))

In [2]:
import cleaning_the_opening_time_format as ctotf


In [3]:
path= '../data/merged_data.csv'
df = ctotf.load_data(path)

INFO:cleaning_the_opening_time_format:Loading the data


In [4]:
df.head()

,Name,Type of Activity,Description,Location,OpeningTimes
0,Al Azhar Park,Outdoor Activity,A large park with beautiful gardens and recrea...,Cairo,N/A
1,Aquarium Grotto Park,Outdoor Activity,A park featuring an aquarium and scenic views.,"Zamalek, Cairo",N/A
2,Family Park,Outdoor Activity,A family-friendly park with various activities.,New Cairo,N/A
3,Nile Cruise (Felucca),Outdoor Activity,Sail the Nile River at sunset.,"Maadi, Zamalek",N/A
4,Zawya,Movie Screenings & Art Performances,Art-house cinema screening alternative films a...,"15 Emad El-Deen, Downtown",Check weekly schedule


### Extracting features like opening and Closing time

In [5]:
df[['OpeningTime', 'ClosingTime']] = df.apply(ctotf.process_opening_times, axis=1)

INFO:cleaning_the_opening_time_format:Processing the OpeningTimes column
INFO:cleaning_the_opening_time_format:Processing the OpeningTimes column
INFO:cleaning_the_opening_time_format:Processing the OpeningTimes column
INFO:cleaning_the_opening_time_format:Processing the OpeningTimes column
INFO:cleaning_the_opening_time_format:Processing the OpeningTimes column
INFO:cleaning_the_opening_time_format:Processing the OpeningTimes column
INFO:cleaning_the_opening_time_format:Processing the OpeningTimes column
INFO:cleaning_the_opening_time_format:Processing the OpeningTimes column
INFO:cleaning_the_opening_time_format:Processing the OpeningTimes column
INFO:cleaning_the_opening_time_format:Processing the OpeningTimes column
INFO:cleaning_the_opening_time_format:Processing the OpeningTimes column
INFO:cleaning_the_opening_time_format:Processing the OpeningTimes column
INFO:cleaning_the_opening_time_format:Processing the OpeningTimes column
INFO:cleaning_the_opening_time_format:Processing th

### Extracing a new feature which is called Duration

In [6]:
df['Duration'] = df.apply(ctotf.calculate_duration,axis=1)

INFO:cleaning_the_opening_time_format:Creating a new feature: Duration (in hours)
INFO:cleaning_the_opening_time_format:Creating a new feature: Duration (in hours)
INFO:cleaning_the_opening_time_format:Creating a new feature: Duration (in hours)
INFO:cleaning_the_opening_time_format:Creating a new feature: Duration (in hours)
INFO:cleaning_the_opening_time_format:Creating a new feature: Duration (in hours)
INFO:cleaning_the_opening_time_format:Creating a new feature: Duration (in hours)
INFO:cleaning_the_opening_time_format:Creating a new feature: Duration (in hours)
ERROR:cleaning_the_opening_time_format:Error occurred while calculating duration: strptime() argument 1 must be str, not None
INFO:cleaning_the_opening_time_format:Creating a new feature: Duration (in hours)
INFO:cleaning_the_opening_time_format:Creating a new feature: Duration (in hours)
INFO:cleaning_the_opening_time_format:Creating a new feature: Duration (in hours)
INFO:cleaning_the_opening_time_format:Creating a new f

### Flaging entries with special schedules

In [7]:
df = ctotf.flag_entries_with_special_schedules(df)

INFO:cleaning_the_opening_time_format:Flagging entries with special schedules


### final ceaning : Replace Nan or 'UnKnown' entries with None for Consistancy

In [8]:
df = ctotf.final_cleaning(df)

INFO:cleaning_the_opening_time_format:Final cleaning: Replace NaN or 'Unknown' entries with None for consistency


In [9]:
df

,Name,Type of Activity,Description,Location,OpeningTimes,OpeningTime,ClosingTime,Duration,Is_Special_Schedule
0,Al Azhar Park,Outdoor Activity,A large park with beautiful gardens and recrea...,Cairo,N/A,None,None,NaN,False
1,Aquarium Grotto Park,Outdoor Activity,A park featuring an aquarium and scenic views.,"Zamalek, Cairo",N/A,None,None,NaN,False
2,Family Park,Outdoor Activity,A family-friendly park with various activities.,New Cairo,N/A,None,None,NaN,False
3,Nile Cruise (Felucca),Outdoor Activity,Sail the Nile River at sunset.,"Maadi, Zamalek",N/A,None,None,NaN,False
4,Zawya,Movie Screenings & Art Performances,Art-house cinema screening alternative films a...,"15 Emad El-Deen, Downtown",Check weekly schedule,None,None,NaN,True
...,...,...,...,...,...,...,...,...,...
60,Climb the Cairo Tower,Outdoor Activity,Enjoy panoramic views of Cairo from the top of...,Cairo,8:00 AM - midnight,08:00,None,NaN,False
61,Spend a Sweet Afternoon at Sugar Spell,Indoor Activity,"A candy shop with arts and crafts, games, and ...",Cairo,N/A,None,None,NaN,False
62,Experience the Calm in a Felucca,Outdoor Activity,Sail on the Nile River in a traditional boat.,"Nile River, Cairo",N/A,None,None,NaN,False
63,Go for a Magical Adventure at Magic Galaxy,Indoor Activity,An indoor amusement center with rides and games.,"City Stars Mall, Cairo",1:00 PM - 11:00 PM,13:00,23:00,10.0,False


In [10]:
df['OpeningTime'].value_counts()

OpeningTime
10:00               10
12:00                7
11:00                6
15:00                5
13:00                3
16:00                3
14:30                1
17:00                1
11:30                1
20:00                1
Special Schedule     1
08:00                1
Name: count, dtype: int64

In [11]:
import datetime as dt
def convert_to_24hr(time_str):
    try:
        # Try to handle time with minutes, e.g. "2:30 PM"
        return dt.datetime.strptime(time_str.strip(), "%I:%M %p").strftime("%H:%M")
    except ValueError:
        try:
            # Handle time without minutes, e.g. "2 PM"
            return dt.datetime.strptime(time_str.strip(), "%I %p").strftime("%H:%M")
        except ValueError:
            try:
                # Handle time without space, e.g. "2PM"
                return dt.datetime.strptime(time_str.strip(), "%I%p").strftime("%H:%M")
            except ValueError:
                return None

In [12]:
time_str = '2:30 PM - 12 AM' 
times = time_str.split('-')
times

['2:30 PM ', ' 12 AM']

In [13]:
x = convert_to_24hr(times[0])
x

'14:30'

In [14]:
df.to_csv('../data/preprocessed_data.csv' , index=False)

In [15]:
df.head()

,Name,Type of Activity,Description,Location,OpeningTimes,OpeningTime,ClosingTime,Duration,Is_Special_Schedule
0,Al Azhar Park,Outdoor Activity,A large park with beautiful gardens and recrea...,Cairo,N/A,None,None,NaN,False
1,Aquarium Grotto Park,Outdoor Activity,A park featuring an aquarium and scenic views.,"Zamalek, Cairo",N/A,None,None,NaN,False
2,Family Park,Outdoor Activity,A family-friendly park with various activities.,New Cairo,N/A,None,None,NaN,False
3,Nile Cruise (Felucca),Outdoor Activity,Sail the Nile River at sunset.,"Maadi, Zamalek",N/A,None,None,NaN,False
4,Zawya,Movie Screenings & Art Performances,Art-house cinema screening alternative films a...,"15 Emad El-Deen, Downtown",Check weekly schedule,None,None,NaN,True


In [16]:
df['Is_Special_Schedule'].value_counts()

Is_Special_Schedule
False    61
True      4
Name: count, dtype: int64

In [17]:
df['OpeningTimes'].value_counts()

OpeningTimes
N/A                                                     19
10 AM - 11 PM                                            7
4 PM - 12 AM                                             3
3 PM - 1 AM                                              2
10 AM - 12 AM                                            2
12 PM - 12 AM                                            2
3 PM - 12 AM                                             2
Saturdays, Mondays & Wednesdays, 7:30 PM - 9:00 PM       1
2:30 PM - 12 AM                                          1
11 AM - 5 PM (Sun-Wed)                                   1
12 PM - 9 PM                                             1
12 PM - 10 PM                                            1
12 PM - 8 PM                                             1
Check schedule on Facebook                               1
3 PM - 3 AM                                              1
11:00 AM - 11:00 PM                                      1
Check weekly schedule                      

In [18]:
import pandas as pd
df2 = pd.read_csv('../data/preprocessed_data - preprocessed_data.csv')

In [19]:
df2.head()

,Name,Type of Activity,Description,Location,OpeningTimes,OpeningTime,ClosingTime,Duration,Is_Special_Schedule,Fee_Adult_EGP,Fee_kid_EGP
0,Al Azhar Park,Outdoor Activity,A large park with beautiful gardens and recrea...,Cairo,9:00 AM -10:00 PM,9:00,22:00,13.0,False,10.0,NaN
1,Aquarium Grotto Park,Outdoor Activity,A park featuring an aquarium and scenic views.,"Zamalek, Cairo",9:00 AM - 5:30 PM,9:00,17:30,8.5,False,20.0,NaN
2,Family Park,Outdoor Activity,A family-friendly park with various activities.,New Cairo,9:00 AM -12:00 AM,9:00,0:00,21.0,False,160.0,460.0
3,Nile Cruise (Felucca),Outdoor Activity,Sail the Nile River at sunset.,"Maadi, Zamalek",9:00 AM -12:00 AM,9:00,0:00,21.0,False,24500.0,NaN
4,Zawya,Movie Screenings & Art Performances,Art-house cinema screening alternative films a...,"15 Emad El-Deen, Downtown",Check weekly schedule,NaN,NaN,NaN,True,100.0,NaN


In [21]:
df2['Fee_Adult_EGP'].isnull().sum()

np.int64(17)

In [22]:
df2.shape

(65, 11)

In [24]:
df2.isnull().sum()

Name                    0
Type of Activity        0
Description             0
Location                0
OpeningTimes            0
OpeningTime             3
ClosingTime             3
Duration                4
Is_Special_Schedule     0
Fee_Adult_EGP          17
Fee_kid_EGP            26
dtype: int64

In [28]:
df2['Location'].value_counts()

Location
Cairo                                                            5
Cairo Festival City Mall                                         2
Giza, Cairo                                                      2
New Cairo                                                        1
Maadi, Zamalek                                                   1
12 Mohamed Anis St., Zamalek                                     1
15 Emad El-Deen, Downtown                                        1
Islamic Cairo                                                    1
Mall of Egypt, 6th of October City                               1
Obour City                                                       1
Zamalek, Cairo                                                   1
The Field Maadi Sports Community Club                            1
Point 90 Mall, New Cairo & Avenue Mall, Obour City               1
Cairo Festival City, 5th Settlement                              1
Inside Mazar Mall, 6th of October City               